In [1]:
import cv2
import numpy as np
import os
from scipy.optimize import least_squares
from tomlkit import boolean
from tqdm import tqdm
import matplotlib.pyplot as plt

from hw3_defs import get_all_matches, normalize_images, grayscale_images, find_features

In [2]:
class Image_loader():
    def __init__(self, img_dir:str, downscale_factor:float, K):
        # loading the Camera intrinsic parameters K
        # with open(img_dir + '/K.txt') as f:
        #     self.K = np.array(list((map(lambda x:list(map(lambda x:float(x), x.strip().split(' '))),f.read().split('\n')))))
        #     self.image_list = []
        self.K = K
        self.image_list = []
        # Loading the set of images
        for image in sorted(os.listdir(img_dir)):
            if image[-4:].lower() == '.jpg' or image[-4:].lower() == '.png':
                self.image_list.append(img_dir + '/' + image)
        
        self.path = os.getcwd()
        self.factor = downscale_factor
        self.downscale()

    
    def downscale(self) -> None:
        '''
        Downscales the Image intrinsic parameter acc to the downscale factor
        '''
        self.K[0, 0] /= self.factor
        self.K[1, 1] /= self.factor
        self.K[0, 2] /= self.factor
        self.K[1, 2] /= self.factor
    
    def downscale_image(self, image):
        for _ in range(1,int(self.factor / 2) + 1):
            image = cv2.pyrDown(image)
        return image

In [3]:
import sys
from os import path as os_path
file_path = os_path.dirname(os_path.realpath(''))
img_dir = file_path + "/HW4/buddha_images"
downscale_factor = 2.0


img1 = cv2.cvtColor(cv2.imread("buddha_images/buddha_005.png"),cv2.COLOR_BGR2RGB)
img_h = img1.shape[1]
img_w = img1.shape[0]
# Camera intrinsics
K = np.array([[img_w, 0, img_w/2],
            [0, img_w, img_h/2],
            [0,0,1]])


# K = np.array([[2759.48, 0, 1520.69], 
#             [0, 2764.16, 1006.81],
#             [0, 0, 1, ]])

img_obj = Image_loader(img_dir,downscale_factor,K)

In [4]:
import glob
import itertools
# Save images in a list
imgs = []
dir_name = 'buddha_images/'
list_of_files = sorted( filter( os.path.isfile, glob.glob(dir_name + '*')))
print(list_of_files[0])
for file_path1 in list_of_files: imgs.append(cv2.cvtColor(cv2.imread(file_path1),cv2.COLOR_BGR2RGB))

imgs_norm = normalize_images(imgs)
imgs_gray = grayscale_images(imgs_norm)
img_h, img_w = imgs_gray[0].shape

buddha_images/buddha_001.png


In [5]:
# Same Hyperparameter settings as HW3
sift = cv2.SIFT_create(nfeatures=5000, nOctaveLayers=16, contrastThreshold=0.025, edgeThreshold=10, sigma=1.4)

kp, des, imgs_sift = find_features(sift, imgs_gray)

ptsList, goodMatchList = get_all_matches(kp, des, print_matches=False)

In [6]:





def optimal_reprojection_error(obj_points) -> np.array:
    '''
    calculates of the reprojection error during bundle adjustment
    returns error 
    '''
    transform_matrix = obj_points[0:12].reshape((3,4))
    K = obj_points[12:21].reshape((3,3))
    rest = int(len(obj_points[21:]) * 0.4)
    p = obj_points[21:21 + rest].reshape((2, int(rest/2))).T
    obj_points = obj_points[21 + rest:].reshape((int(len(obj_points[21 + rest:])/3), 3))
    rot_matrix = transform_matrix[:3, :3]
    tran_vector = transform_matrix[:3, 3]
    rot_vector, _ = cv2.Rodrigues(rot_matrix)
    image_points, _ = cv2.projectPoints(obj_points, rot_vector, tran_vector, K, None)
    image_points = image_points[:, 0, :]
    error = [ (p[idx] - image_points[idx])**2 for idx in range(len(p))]
    return np.array(error).ravel()/len(p)

def bundle_adjustment(_3d_point, opt, transform_matrix_new, K, r_error) -> tuple:
    '''
    Bundle adjustment for the image and object points
    returns object points, image points, transformation matrix
    '''
    opt_variables = np.hstack((transform_matrix_new.ravel(), K.ravel()))
    opt_variables = np.hstack((opt_variables, opt.ravel()))
    opt_variables = np.hstack((opt_variables, _3d_point.ravel()))

    values_corrected = least_squares(optimal_reprojection_error, opt_variables, gtol = r_error).x
    K = values_corrected[12:21].reshape((3,3))
    rest = int(len(values_corrected[21:]) * 0.4)
    return values_corrected[21 + rest:].reshape((int(len(values_corrected[21 + rest:])/3), 3)), values_corrected[21:21 + rest].reshape((2, int(rest/2))).T, values_corrected[0:12].reshape((3,4))

def to_ply(path, point_cloud, colors) -> None:
    '''
    Generates the .ply which can be used to open the point cloud
    '''
    out_points = point_cloud.reshape(-1, 3) * 200
    out_colors = colors.reshape(-1, 3)
    print(out_colors.shape, out_points.shape)
    verts = np.hstack([out_points, out_colors])


    mean = np.mean(verts[:, :3], axis=0)
    scaled_verts = verts[:, :3] - mean
    dist = np.sqrt(scaled_verts[:, 0] ** 2 + scaled_verts[:, 1] ** 2 + scaled_verts[:, 2] ** 2)
    indx = np.where(dist < np.mean(dist) + 300)
    verts = verts[indx]
    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar blue
        property uchar green
        property uchar red
        end_header
        '''
    with open(path + '/HW4/res1/' + 'myPC1.ply', 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')




def find_features1(image_0, image_1) -> tuple:
    '''
    Feature detection using the sift algorithm and KNN
    return keypoints(features) of image1 and image2
    '''

    # Same Hyperparameter settings as HW3
    sift = cv2.SIFT_create(nfeatures=5000, nOctaveLayers=16, contrastThreshold=0.025, edgeThreshold=10, sigma=1.4)

    # sift = cv2.SIFT_create()
    key_points_0, desc_0 = sift.detectAndCompute(cv2.cvtColor(image_0, cv2.COLOR_BGR2GRAY), None)
    key_points_1, desc_1 = sift.detectAndCompute(cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY), None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50) # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params,search_params)

    matches = flann.knnMatch(desc_0,desc_1,k=2)

    # bf = cv2.BFMatcher()
    # matches = bf.knnMatch(desc_0, desc_1, k=2)
    feature = []
    for m, n in matches:
        if m.distance < 0.70 * n.distance:
            feature.append(m)

    return np.float32([key_points_0[m.queryIdx].pt for m in feature]), np.float32([key_points_1[m.trainIdx].pt for m in feature])


In [7]:
enable_bundle_adjustment = False

# Find Feature Matches

In [79]:
transform_matrix_0 = np.eye(4)[:3,:]
pose_array = img_obj.K.ravel()

pose_0 = np.matmul(img_obj.K, transform_matrix_0)
pose_1 = np.empty((3, 4)) 
total_points = np.zeros((1, 3))
total_colors = np.zeros((1, 3))

image_0 = img_obj.downscale_image(cv2.imread(img_obj.image_list[0]))
image_1 = img_obj.downscale_image(cv2.imread(img_obj.image_list[1]))

feature_0, feature_1 = find_features1(image_0, image_1)

In [ ]:
total_images = len(img_obj.image_list) - 2 

# for img in imgs_gray:
#     k, d = sift_object.detectAndCompute(img,None)
#     img_sift = cv2.drawKeypoints(img, k, img, color=[255,255,0], flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#     imgs_sift.append(img_sift)
#     kp.append(np.array(k))
#     des.append(np.array(d))

for i in tqdm(range(total_images)):
    # Downscale the Image
    image_2 = img_obj.downscale_image(cv2.imread(img_obj.image_list[i + 2]))
    # Find and Match Features
    features_cur, features_2 = find_features1(image_1, image_2)
    
    image_0 = np.copy(image_1)
    image_1 = np.copy(image_2)
    feature_0 = np.copy(features_cur)
    feature_1 = np.copy(features_2)

# Find Essential Matrix

In [80]:
def get_essential_matrix(points1, points2):
    # Essential matrix
    essential_matrix, em_mask = cv2.findEssentialMat(points1, points2, img_obj.K, method=cv2.RANSAC, prob=0.999, threshold=0.4, mask=None)
    # Only keep inlier points
    points1 = feature_0[em_mask.ravel() == 1]
    points2 = feature_1[em_mask.ravel() == 1]

    return essential_matrix, em_mask, (points1, points2)

In [81]:
essential_matrix, em_mask, (feature_0, feature_1) = get_essential_matrix(feature_0, feature_1)

# Recover Pose
Recover Pose from the essential matrix using cv2.recoverPose.<br>
This method returns triangulated 3D points that pass the cheirality check

In [112]:
def get_pose(essential_matrix, em_mask, pts1, pts2):
    _, rot_matrix, tran_matrix, em_mask = cv2.recoverPose(essential_matrix, pts1, pts2, img_obj.K)

    transformation_matrix = np.hstack((rot_matrix, tran_matrix))

    # Apply Cheirality check to points
    f_pts1 = feature_0[em_mask.ravel() > 0]
    f_pts2 = feature_1[em_mask.ravel() > 0]

    return transformation_matrix, f_pts1, f_pts2

In [113]:
transformation_matrix_1, feature_0, feature_1 = get_pose(essential_matrix, em_mask, feature_0, feature_1)

# Triangulation

In [114]:
def triangulation(point_2d_1, point_2d_2, projection_matrix_1, projection_matrix_2) -> tuple:
    '''
    Triangulates 3d points from 2d vectors and projection matrices
    returns projection matrix of first camera, projection matrix of second camera, point cloud 
    '''
    pt_cloud = cv2.triangulatePoints(point_2d_1, point_2d_2, projection_matrix_1.T, projection_matrix_2.T)
    return projection_matrix_1.T, projection_matrix_2.T, (pt_cloud / pt_cloud[3])    


In [115]:
pose_1 = np.matmul(img_obj.K, transformation_matrix_1)

feature_0_new, feature_1_new, points_3d = triangulation(pose_0, pose_1, feature_0, feature_1)

# Reprojection Error

In [116]:
def reprojection_error(obj_points, image_points, transform_matrix, K, homogenity) ->tuple:
    '''
    Calculates the reprojection error ie the distance between the projected points and the actual points.
    returns total error, object points
    '''
    rot_matrix = transform_matrix[:3, :3]
    tran_vector = transform_matrix[:3, 3]
    rot_vector, _ = cv2.Rodrigues(rot_matrix)
    if homogenity == 1:
        obj_points = cv2.convertPointsFromHomogeneous(obj_points.T)
    image_points_calc, _ = cv2.projectPoints(obj_points, rot_vector, tran_vector, K, None)
    image_points_calc = np.float32(image_points_calc[:, 0, :])
    total_error = cv2.norm(image_points_calc, np.float32(image_points.T) if homogenity == 1 else np.float32(image_points), cv2.NORM_L2)
    return total_error / len(image_points_calc), obj_points

In [117]:
error, points_3d_new = reprojection_error(points_3d, feature_1_new, transformation_matrix_1, img_obj.K, homogenity = 1)
#ideally error < 1
print("REPROJECTION ERROR: ", error)

REPROJECTION ERROR:  0.04128561372247956


# PnP

In [118]:
def PnP(obj_point, image_point , K, dist_coeff, rot_vector, initial) ->  tuple:
    '''
    Finds an object pose from 3D-2D point correspondences using the RANSAC scheme.
    returns rotational matrix, translational matrix, image points, object points, rotational vector
    '''
    if initial == 1:
        obj_point = obj_point[:, 0 ,:]
        image_point = image_point.T
        rot_vector = rot_vector.T 
    _, rot_vector_calc, tran_vector, inlier = cv2.solvePnPRansac(obj_point, image_point, K, dist_coeff, cv2.SOLVEPNP_ITERATIVE)
    # Converts a rotation matrix to a rotation vector or vice versa
    rot_matrix, _ = cv2.Rodrigues(rot_vector_calc)

    if inlier is not None:
        image_point = image_point[inlier[:, 0]]
        obj_point = obj_point[inlier[:, 0]]
        rot_vector = rot_vector[inlier[:, 0]]
    return rot_matrix, tran_vector, image_point, obj_point, rot_vector

In [119]:
_, _, feature_1_pnp, points_3d_pnp, _ = PnP(points_3d_new, feature_1_new, img_obj.K, np.zeros((5, 1), dtype=np.float32), feature_0_new, initial=1)

# Loop for all Images

In [110]:
def common_points(image_points_1, image_points_2, image_points_3) -> tuple:
    '''
    Finds the common points between image 1 and 2 , image 2 and 3
    returns common points of image 1-2, common points of image 2-3, mask of common points 1-2 , mask for common points 2-3 
    '''
    cm_points_1 = []
    cm_points_2 = []
    for i in range(image_points_1.shape[0]):
        a = np.where(image_points_2 == image_points_1[i, :])
        if a[0].size != 0:
            cm_points_1.append(i)
            cm_points_2.append(a[0][0])

    mask_array_1 = np.ma.array(image_points_2, mask=False)
    mask_array_1.mask[cm_points_2] = True
    mask_array_1 = mask_array_1.compressed()
    mask_array_1 = mask_array_1.reshape(int(mask_array_1.shape[0] / 2), 2)

    mask_array_2 = np.ma.array(image_points_3, mask=False)
    mask_array_2.mask[cm_points_2] = True
    mask_array_2 = mask_array_2.compressed()
    mask_array_2 = mask_array_2.reshape(int(mask_array_2.shape[0] / 2), 2)
    print(" Shape New Array", mask_array_1.shape, mask_array_2.shape)
    return np.array(cm_points_1), np.array(cm_points_2), mask_array_1, mask_array_2

In [ ]:


total_images = len(img_obj.image_list) - 2 
pose_array = np.hstack((np.hstack((pose_array, pose_0.ravel())), pose_1.ravel()))

threshold = 0.5
for i in tqdm(range(total_images)):
    # Downscale the Image
    image_2 = img_obj.downscale_image(cv2.imread(img_obj.image_list[i + 2]))
    # Find and Match Features
    features_cur, features_2 = find_features1(image_1, image_2)
    
    if i != 0:
        # Perform Triangulation to get 3D points
        feature_0, feature_1, points_3d = triangulation(pose_0, pose_1, feature_0, feature_1)
        feature_1 = feature_1.T
        points_3d = cv2.convertPointsFromHomogeneous(points_3d.T)
        points_3d = points_3d[:, 0, :]
    
    # Finds common points between images
    cm_points_0, cm_points_1, cm_mask_0, cm_mask_1 = common_points(feature_1, features_cur, features_2)
    cm_points_2 = features_2[cm_points_1]
    cm_points_cur = features_cur[cm_points_1]

    # Performs PnP
    rot_matrix, tran_matrix, cm_points_2, points_3d, cm_points_cur = PnP(points_3d[cm_points_0], cm_points_2, img_obj.K, np.zeros((5, 1), dtype=np.float32), cm_points_cur, initial = 0)
    transform_matrix_1 = np.hstack((rot_matrix, tran_matrix))
    pose_2 = np.matmul(img_obj.K, transform_matrix_1)

    error, points_3d = reprojection_error(points_3d, cm_points_2, transform_matrix_1, img_obj.K, homogenity = 0)

    
    cm_mask_0, cm_mask_1, points_3d = triangulation(pose_1, pose_2, cm_mask_0, cm_mask_1)
    error, points_3d = reprojection_error(points_3d, cm_mask_1, transform_matrix_1, img_obj.K, homogenity = 1)
    print("Reprojection Error: ", error)
    pose_array = np.hstack((pose_array, pose_2.ravel()))
    # takes a long time to run
    if enable_bundle_adjustment:
        points_3d, cm_mask_1, transform_matrix_1 = bundle_adjustment(points_3d, cm_mask_1, transform_matrix_1, img_obj.K, threshold)
        pose_2 = np.matmul(img_obj.K, transform_matrix_1)
        error, points_3d = reprojection_error(points_3d, cm_mask_1, transform_matrix_1, img_obj.K, homogenity = 0)
        print("Bundle Adjusted error: ",error)
        total_points = np.vstack((total_points, points_3d))
        points_left = np.array(cm_mask_1, dtype=np.int32)
        color_vector = np.array([image_2[l[1], l[0]] for l in points_left])
        total_colors = np.vstack((total_colors, color_vector))
    else:
        total_points = np.vstack((total_points, points_3d[:, 0, :]))
        points_left = np.array(cm_mask_1, dtype=np.int32)
        color_vector = np.array([image_2[l[1], l[0]] for l in points_left.T])
        total_colors = np.vstack((total_colors, color_vector)) 



    transform_matrix_0 = np.copy(transform_matrix_1)
    pose_0 = np.copy(pose_1)
    # plt.scatter(i, error)
    # plt.pause(0.05)

    image_0 = np.copy(image_1)
    image_1 = np.copy(image_2)
    feature_0 = np.copy(features_cur)
    feature_1 = np.copy(features_2)
    pose_1 = np.copy(pose_2)
    #cv2.imshow(img_obj.image_list[0].split('/')[-2], image_2)
    # if cv2.waitKey(1) & 0xff == ord('q'):
    #     break

print("Printing to .ply file")
print(total_points.shape, total_colors.shape)
to_ply(file_path, total_points, total_colors)
print("Completed Exiting ...")
np.savetxt(file_path + '/HW4/res1/' + 'my1_pose_array.csv', pose_array, delimiter = '\n')


 10%|█         | 2/20 [00:00<00:01,  9.79it/s]

 Shape New Array (55, 2) (55, 2)
Reprojection Error:  7.937869808120239
 Shape New Array (49, 2) (49, 2)
Reprojection Error:  4.522023785147788


 20%|██        | 4/20 [00:00<00:01,  9.74it/s]

 Shape New Array (64, 2) (64, 2)
Reprojection Error:  9.853451254103154
 Shape New Array (50, 2) (50, 2)
Reprojection Error:  5.825446933665786


 30%|███       | 6/20 [00:00<00:01,  9.48it/s]

...

 Shape New Array (61, 2) (61, 2)
Reprojection Error:  40.570982585798305
 Shape New Array (61, 2) (61, 2)
Reprojection Error:  14.057082394965633


 90%|█████████ | 18/20 [00:01<00:00,  9.17it/s]

 Shape New Array (75, 2) (75, 2)
Reprojection Error:  36.46435462653994
 Shape New Array (63, 2) (63, 2)
Reprojection Error:  5.294938330733784


100%|██████████| 20/20 [00:02<00:00,  9.43it/s]

 Shape New Array (56, 2) (56, 2)
Reprojection Error:  43.41064897072237
 Shape New Array (40, 2) (40, 2)
Reprojection Error:  2.7932453086241433
Printing to .ply file
(1124, 3) (1124, 3)
(1124, 3) (1124, 3)
Completed Exiting ...


In [ ]:
import open3d as o3d
pcd = o3d.io.read_point_cloud(file_path + '/HW4/res1/myPC1.ply')
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

[Open3D WARNING] Read PLY failed: unable to read file: /home/meuli/src/eece7150/HW4/res1/myPC1.ply


RPly: Aborted by user
